In [1]:
import os
import json
import numpy as np

root_dir='/home/aimotion/Desktop/Radiate/dataset/dataset_original/'

In [2]:
def gen_boundingbox(bbox, angle):
    theta = np.deg2rad(-angle)
    R = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    points = np.array([[bbox[0], bbox[1]],
                       [bbox[0] + bbox[2], bbox[1]],
                       [bbox[0] + bbox[2], bbox[1] + bbox[3]],
                       [bbox[0], bbox[1] + bbox[3]]]).T
    
    cx = bbox[0] + bbox[2] / 2
    cy = bbox[1] + bbox[3] / 2
    T = np.array([[cx], [cy]])
    
    points = points - T
    points = np.matmul(R, points) + T
    points = points.astype(int)

    min_x = np.min(points[0, :])
    min_y = np.min(points[1, :])
    max_x = np.max(points[0, :])
    max_y = np.max(points[1, :])
    
    return min_x, min_y, max_x, max_y

In [3]:
def get_radar_dicts(folders):
    dataset_dicts = []
    idd = 0
    folder_size = len(folders)
    
    for folder in folders:
        radar_folder = os.path.join(root_dir, folder, 'Navtech_Cartesian')
        annotation_path = os.path.join(root_dir, folder, 'annotations', 'annotations.json')
        
        with open(annotation_path, 'r') as f_annotation:
            annotation = json.load(f_annotation)
        
        radar_files = os.listdir(radar_folder)
        radar_files.sort()
        
        for frame_number in range(len(radar_files)):
            record = {}
            objs = []
            bb_created = False
            idd += 1
            filename = os.path.join(radar_folder, radar_files[frame_number])
            
            if (not os.path.isfile(filename)):
                print(filename)
                continue
            record["file_name"] = filename
            record["image_id"] = idd
            
            for object in annotation:
                if (object['bboxes'][frame_number]):
                    class_obj = object['class_name']
                    
                    if (class_obj != 'pedestrian' and class_obj != 'group_of_pedestrians'):
                        bbox = object['bboxes'][frame_number]['position']
                        angle = object['bboxes'][frame_number]['rotation']
                        bb_created = True
                        xmin, ymin, xmax, ymax = gen_boundingbox(bbox, angle)
                        obj = {
                                "bbox": [xmin, ymin, xmax, ymax]
                        }
                        objs.append(obj)
                                
            if bb_created:
                record["annotations"] = objs
                dataset_dicts.append(record)
    return dataset_dicts

In [20]:
folders=['junction_1_7']
print(len(get_radar_dicts(folders)))
print(get_radar_dicts(folders))

323
[{'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000158.png', 'image_id': 158, 'annotations': [{'bbox': [518, 17, 545, 47]}]}, {'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000159.png', 'image_id': 159, 'annotations': [{'bbox': [535, 35, 563, 65]}]}, {'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000160.png', 'image_id': 160, 'annotations': [{'bbox': [530, 56, 558, 85]}]}, {'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000161.png', 'image_id': 161, 'annotations': [{'bbox': [537, 76, 565, 106]}]}, {'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000162.png', 'image_id': 162, 'annotations': [{'bbox': [520, 97, 548, 127]}]}, {'file_name': '/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_C

In [5]:
# Create xml 
from xml.etree.ElementTree import Element, SubElement, ElementTree, dump

def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [6]:
# From the beginning
number_of_files = len(get_radar_dicts(folders))

for i in range (number_of_files):
    list_1 = get_radar_dicts(folders)[i]['file_name']
    file_name = str(list_1)

    root = Element('annotation')
    head = SubElement(root, "file_name").text = file_name
    head = SubElement(root, "image_id").text = str(i)
    head = SubElement(root, "weather").text = 'sunny'
    
    head = SubElement(root, "size")
    SubElement(head, "width").text = "1152"
    SubElement(head, "height").text = "1152"
    
    number_of_objects = len(get_radar_dicts(folders)[i]['annotations'])
    
    for ob in range (number_of_objects):
        head = SubElement(root, "object")
        head1 = SubElement(head, "name").text = "vehicle"
        head2 = SubElement(head, "bndbox")
        SubElement(head2, "xmin").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][0])
        SubElement(head2, "ymin").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][1])
        SubElement(head2, "xmax").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][2])
        SubElement(head2, "ymax").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][3])
    
    indent(root)   
    dump(root)     
    
    tree = ElementTree(root)
    tree.write('/home/aimotion/Desktop/Radiate/dataset/dataset_xml/sunny2/annotations/'+'sunny_'+str(i)+'.xml')

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000010.png</file_name>
  <image_id>0</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>527</xmin>
      <ymin>1</ymin>
      <xmax>556</xmax>
      <ymax>37</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000011.png</file_name>
  <image_id>1</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>535</xmin>
      <ymin>18</ymin>
      <xmax>564</xmax>
      <ymax>55</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000012.png</file

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000028.png</file_name>
  <image_id>18</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>542</xmin>
      <ymin>304</ymin>
      <xmax>568</xmax>
      <ymax>341</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>513</xmin>
      <ymin>941</ymin>
      <xmax>535</xmax>
      <ymax>976</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000029.png</file_name>
  <image_id>19</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>542</xmin>
      <ymin>320</ymin>
      <xmax>568</xmax>
      <ymax>357</y

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000042.png</file_name>
  <image_id>32</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>550</xmin>
      <ymin>534</ymin>
      <xmax>568</xmax>
      <ymax>569</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>536</xmin>
      <ymin>637</ymin>
      <xmax>556</xmax>
      <ymax>676</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000043.png</file_name>
  <image_id>33</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>550</xmin>
      <ymin>545</ymin>
      <xmax>568</xmax>
      <ymax>579</y

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000056.png</file_name>
  <image_id>46</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>545</xmin>
      <ymin>772</ymin>
      <xmax>563</xmax>
      <ymax>807</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>537</xmin>
      <ymin>286</ymin>
      <xmax>558</xmax>
      <ymax>330</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000057.png</file_name>
  <image_id>47</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>545</xmin>
      <ymin>790</ymin>
      <xmax>563</xmax>
      <ymax>825</y

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000072.png</file_name>
  <image_id>62</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>509</xmin>
      <ymin>1037</ymin>
      <xmax>539</xmax>
      <ymax>1076</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000073.png</file_name>
  <image_id>63</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>514</xmin>
      <ymin>1055</ymin>
      <xmax>545</xmax>
      <ymax>1093</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/00007

...

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000522.png</file_name>
  <image_id>355</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>536</xmin>
      <ymin>821</ymin>
      <xmax>560</xmax>
      <ymax>932</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>551</xmin>
      <ymin>598</ymin>
      <xmax>565</xmax>
      <ymax>626</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>533</xmin>
      <ymin>476</ymin>
      <xmax>548</xmax>
      <ymax>511</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>529</xmin>
      <ymin>116</ymin>
      <xmax>568</xmax>
      <ymax>180</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/da

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000530.png</file_name>
  <image_id>363</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>509</xmin>
      <ymin>944</ymin>
      <xmax>552</xmax>
      <ymax>1039</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>549</xmin>
      <ymin>735</ymin>
      <xmax>564</xmax>
      <ymax>763</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>528</xmin>
      <ymin>300</ymin>
      <xmax>547</xmax>
      <ymax>329</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>547</xmin>
      <ymin>297</ymin>
      <xmax>574</xmax>
      <ymax>361</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>542</xmin>


<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000538.png</file_name>
  <image_id>371</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>489</xmin>
      <ymin>1068</ymin>
      <xmax>530</xmax>
      <ymax>1151</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>539</xmin>
      <ymin>866</ymin>
      <xmax>554</xmax>
      <ymax>893</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>551</xmin>
      <ymin>442</ymin>
      <xmax>574</xmax>
      <ymax>530</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>549</xmin>
      <ymin>340</ymin>
      <xmax>573</xmax>
      <ymax>376</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000548.png</file_name>
  <image_id>381</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>546</xmin>
      <ymin>677</ymin>
      <xmax>568</xmax>
      <ymax>740</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>551</xmin>
      <ymin>542</ymin>
      <xmax>569</xmax>
      <ymax>578</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000549.png</file_name>
  <image_id>382</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>544</xmin>
      <ymin>698</ymin>
      <xmax>567</xmax>
      <ymax>760<

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000562.png</file_name>
  <image_id>395</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>516</xmin>
      <ymin>953</ymin>
      <xmax>549</xmax>
      <ymax>1015</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>544</xmin>
      <ymin>827</ymin>
      <xmax>560</xmax>
      <ymax>856</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000563.png</file_name>
  <image_id>396</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>541</xmin>
      <ymin>848</ymin>
      <xmax>558</xmax>
      <ymax>877

In [21]:
# Continue

number_of_files = len(get_radar_dicts(folders))

for i in range (number_of_files):
    
    list_1 = get_radar_dicts(folders)[i]['file_name']
    file_name = str(list_1)

    root = Element('annotation')
    head = SubElement(root, "file_name").text = file_name
    head = SubElement(root, "image_id").text = str(i)
    head = SubElement(root, "weather").text = 'sunny'
    
    head = SubElement(root, "size")
    SubElement(head, "width").text = "1152"
    SubElement(head, "height").text = "1152"
    
    number_of_objects = len(get_radar_dicts(folders)[i]['annotations'])
    
    for ob in range (number_of_objects):
        head = SubElement(root, "object")
        head1 = SubElement(head, "name").text = "vehicle"
        head2 = SubElement(head, "bndbox")
        SubElement(head2, "xmin").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][0])
        SubElement(head2, "ymin").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][1])
        SubElement(head2, "xmax").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][2])
        SubElement(head2, "ymax").text = str(get_radar_dicts(folders)[i]['annotations'][ob]['bbox'][3])
    
    indent(root)   
    dump(root)     
    
    tree = ElementTree(root)
    tree.write('/home/aimotion/Desktop/Radiate/dataset/dataset_xml/sunny2/test/'+'sunny_'+str(i+2663)+'.xml')

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000158.png</file_name>
  <image_id>0</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>518</xmin>
      <ymin>17</ymin>
      <xmax>545</xmax>
      <ymax>47</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000159.png</file_name>
  <image_id>1</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>535</xmin>
      <ymin>35</ymin>
      <xmax>563</xmax>
      <ymax>65</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000160.png</fil

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000179.png</file_name>
  <image_id>21</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>546</xmin>
      <ymin>417</ymin>
      <xmax>568</xmax>
      <ymax>448</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000180.png</file_name>
  <image_id>22</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>549</xmin>
      <ymin>435</ymin>
      <xmax>571</xmax>
      <ymax>466</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000181.pn

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000199.png</file_name>
  <image_id>41</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>548</xmin>
      <ymin>774</ymin>
      <xmax>564</xmax>
      <ymax>802</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000200.png</file_name>
  <image_id>42</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>548</xmin>
      <ymin>795</ymin>
      <xmax>564</xmax>
      <ymax>824</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000201.pn

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000252.png</file_name>
  <image_id>60</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>506</xmin>
      <ymin>1008</ymin>
      <xmax>529</xmax>
      <ymax>1031</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000253.png</file_name>
  <image_id>61</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>511</xmin>
      <ymin>987</ymin>
      <xmax>534</xmax>
      <ymax>1010</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000254

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000268.png</file_name>
  <image_id>76</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>547</xmin>
      <ymin>341</ymin>
      <xmax>572</xmax>
      <ymax>371</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>536</xmin>
      <ymin>672</ymin>
      <xmax>554</xmax>
      <ymax>697</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000269.png</file_name>
  <image_id>77</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>544</xmin>
      <ymin>370</ymin>
      <xmax>570</xmax>
      <ymax>400</y

...

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000634.png</file_name>
  <image_id>240</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>542</xmin>
      <ymin>325</ymin>
      <xmax>572</xmax>
      <ymax>359</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000635.png</file_name>
  <image_id>241</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>540</xmin>
      <ymin>345</ymin>
      <xmax>570</xmax>
      <ymax>379</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000636.

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000654.png</file_name>
  <image_id>260</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>546</xmin>
      <ymin>701</ymin>
      <xmax>564</xmax>
      <ymax>733</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000655.png</file_name>
  <image_id>261</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>546</xmin>
      <ymin>718</ymin>
      <xmax>564</xmax>
      <ymax>750</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000656.

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000674.png</file_name>
  <image_id>280</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>517</xmin>
      <ymin>1022</ymin>
      <xmax>542</xmax>
      <ymax>1049</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000675.png</file_name>
  <image_id>281</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>517</xmin>
      <ymin>1036</ymin>
      <xmax>542</xmax>
      <ymax>1063</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000697.png</file_name>
  <image_id>300</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>525</xmin>
      <ymin>817</ymin>
      <xmax>548</xmax>
      <ymax>847</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000698.png</file_name>
  <image_id>301</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>530</xmin>
      <ymin>801</ymin>
      <xmax>553</xmax>
      <ymax>831</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000699.

<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000717.png</file_name>
  <image_id>320</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>535</xmin>
      <ymin>451</ymin>
      <xmax>552</xmax>
      <ymax>479</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000718.png</file_name>
  <image_id>321</image_id>
  <weather>sunny</weather>
  <size>
    <width>1152</width>
    <height>1152</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>536</xmin>
      <ymin>432</ymin>
      <xmax>554</xmax>
      <ymax>460</ymax>
    </bndbox>
  </object>
</annotation>
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000719.